In [2]:
import asyncio
from fastapi import FastAPI, Form, HTTPException

# Dummy implementation for calc_eta that always returns 30 minutes.
async def calc_eta(start_location: str, destination: str) -> int:
    return 30

# Dummy implementation for scheduled_call that just prints a message.
async def scheduled_call(delay: float, mode: int, trip_data: dict, user_data: dict):
    print(f"Scheduled call (mode {mode}) will occur after {delay} seconds.")
    print(f"Trip data: {trip_data}")
    print(f"User data: {user_data}")

# Dummy Supabase client implementation.
class DummySupabaseTable:
    def __init__(self, table_name: str):
        self.table_name = table_name
    def insert(self, data: dict):
        self.data = data
        return self
    def select(self, *args, **kwargs):
        return self
    def eq(self, key: str, value):
        return self
    def execute(self):
        if self.table_name == "trips":
            # Simulate an insert response with an auto-generated trip id.
            return type("Response", (), {"data": [{
                "id": 123,
                "user_id": self.data.get("user_id"),
                "start_location": self.data.get("start_location"),
                "destination": self.data.get("destination"),
                "status": "in progress",
                "eta": self.data.get("eta")
            }]})
        elif self.table_name == "users":
            # Simulate a user lookup response.
            return type("Response", (), {"data": [{
                "id": 1,
                "phone_num": "+15555555555",
                "first_name": "John",
                "last_name": "Doe"
            }]})
        else:
            return type("Response", (), {"data": []})

class DummySupabase:
    def table(self, table_name: str):
        return DummySupabaseTable(table_name)

# Replace supabase with our dummy version.
supabase = DummySupabase()

# Create a FastAPI app instance.
app = FastAPI()

@app.post("/start")
async def save_trip(
    user_id: int = Form(...), 
    start_location: str = Form(...),
    destination: str = Form(...)
):
    """
    Combines saving a new trip and starting it.
        1. Calculates ETA (with a 20% safety buffer) using Google Maps.
        2. Inserts the trip into the DB with status "in progress" and the calculated ETA.
        3. Retrieves the inserted trip and user details.
        4. Schedules timers (e.g., call at 50% and 120% of ETA).
    """
    eta_minutes = await calc_eta(start_location, destination)
    eta_seconds = eta_minutes * 60

    # Insert trip into DB.
    insert_response = supabase.table("trips").insert({
        "user_id": user_id,
        "start_location": start_location,
        "destination": destination,
        "status": "in progress",
        "eta": eta_minutes
    }).execute()

    if not insert_response.data:
        raise HTTPException(status_code=500, detail="Failed to insert trip")
    
    # Retrieve trip data.
    trip_data = insert_response.data[0]
    trip_id = trip_data["id"]

    user_response = supabase.table("users").select("*").eq("id", user_id).execute()
    if not user_response.data:
        raise HTTPException(status_code=404, detail="User not found")
    user_data = user_response.data[0]

    if user_response.data:
        # Schedule calls.
        asyncio.create_task(
            scheduled_call(eta_seconds * 0.5, 0, trip_data, user_data)
        )
        asyncio.create_task(
            scheduled_call(eta_seconds * 1.2, 1, trip_data, user_data)
        )

        return {
            "trip_id": trip_id,
            "eta": eta_minutes,
            "message": "Trip started successfully"
        }
    else:
        raise HTTPException(status_code=500, detail="Failed to update trip")

# Testing function to call save_trip directly.
async def test_save_trip_directly():
    # Since our endpoint uses Form parameters, calling the function directly bypasses dependency injection.
    # We can directly pass the values as function arguments.
    result = await save_trip(user_id=1, start_location="Toronto", destination="Waterloo")
    print(result)
    return result

# Run the test function (e.g., in an async Jupyter cell).
await test_save_trip_directly()


{'trip_id': 123, 'eta': 30, 'message': 'Trip started successfully'}


{'trip_id': 123, 'eta': 30, 'message': 'Trip started successfully'}

Scheduled call (mode 0) will occur after 900.0 seconds.
Trip data: {'id': 123, 'user_id': 1, 'start_location': 'Toronto', 'destination': 'Waterloo', 'status': 'in progress', 'eta': 30}
User data: {'id': 1, 'phone_num': '+15555555555', 'first_name': 'John', 'last_name': 'Doe'}
Scheduled call (mode 1) will occur after 2160.0 seconds.
Trip data: {'id': 123, 'user_id': 1, 'start_location': 'Toronto', 'destination': 'Waterloo', 'status': 'in progress', 'eta': 30}
User data: {'id': 1, 'phone_num': '+15555555555', 'first_name': 'John', 'last_name': 'Doe'}
